## Aggregating Yelp Restaurant Data
Yelp API Guide: https://docs.developer.yelp.com/docs/fusion-intro. Uses the v3/businesses/search endpoint.

In [ ]:
import json
import os
import requests
import string
import time
import urllib.request
from urllib.error import HTTPError
import unidecode

In [ ]:
# Aggregate Manhattan specific restuarants by neighborhood for more granular results.
neighborhoods = [
    "Alphabet City",
    "Battery Park City",
    "Bowery",
    "Bryant Park",
    "Carnegie Hill",
    "Central Park",
    "Chelsea",
    "Chinatown",
    "Civic Center",
    "Clinton",
    "East Harlem",
    "East Village",
    "Financial District",
    "Flatiron",
    "Fort George",
    "Garment District",
    "Gramercy",
    "Greenwich Village",
    "Hamilton Heights",
    "Harlem",
    "Hells Kitchen",
    "Hudson Heights",
    "Hudson Square",
    "Hudson Yards",
    "Inwood",
    "Kips Bay",
    "Lenox Hill",
    "Lincoln Square",
    "Little Italy",
    "Lower East Side",
    "Manhattan Valley",
    "Manhattanville",
    "Meatpacking",
    "Midtown",
    "Midtown East",
    "Midtown South",
    "Midtown West",
    "Morningside Heights",
    "Murray Hill",
    "Noho",
    "Nolita",
    "NoMad",
    "Roosevelt Island",
    "Soho",
    "Stuyvesant Town",
    "Sutton Place",
    "Times Square",
    "Theater District",
    "Tribeca",
    "Tudor City",
    "Turtle Bay",
    "Two Bridges",
    "Union Square",
    "Upper East Side",
    "Upper West Side",
    "Washington Heights",
    "Washington Square Park",
    "West Harlem",
    "West Village",
    "Yorkville",
]

In [ ]:
# Yelp allows inly 500 API calls per day, so we rotate btwn 3 different keys for maximum data collection.
kKeyIndex = 0
keys = [
    "ZF5VOfPCUWtK2C4_ZMpMrO3FxyS6EGlN_aCjNPBTYZyHhmMZvi7sADCFioEuDUalKlL_83AGB1fWkICmFeHudLzmUhtUq589kgKpnfQbQoT2BMznqTLJ2cIX1RRAZXYx",
    "QOCKsANBYQUN4Fmrxh23mAl5Bjbi69gv3W7ChGNOmp98Q3124aytz9F2MzEPhmKOXa6EomrQAjLeGEZuvlrbsR5Q_KSnsST7Ona_K0_wafErqsrxsd68aCSe9j9IZXYx",
    "NO9vZwZGnE58R8YbQDEPC90SlZ2eok4O4aYkdIxH96vUZMeSCDCvIZYY7L3VxWVYiMITiaMIkOBPRdtOgkR52BwBexnpVDDmhcjWClFRgu8uByoBopPAP8stZUBIZXYx"  
]

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
headers = {
    "accept": "application/json",
    "Authorization": "Bearer " + keys[kKeyIndex]
}

In [ ]:
alias_to_content = {}

In [ ]:
for neighborhood in neighborhoods:
    print("Fetching data for " + neighborhood)
    
    # Maximum results per API request.
    limit = 50
    location = neighborhood + ", Manhattan, NY"
    location = location.replace(" ", "+")
    
    # Get up to 1000 restaurants per neighborhood.
    for i in range(0, 1000, limit):        
        url_params = {
            "location": location,
            "term": "Restaurants",
            "limit": limit,
            "offset": i,
            "categories": "(restaurants, All)",
            "sort_by": "distance",
        }

        response = requests.get(url, headers=headers, params=url_params)
        
        # Max API calls gets a return status == 429!
        if response.status_code == 429: 
            print("Rotating key")
            kKeyIndex += 1
            headers["Authorization"] = "Bearer " + keys[kKeyIndex]
            response = requests.get(url, headers=headers, params=url_params)

        if response.status_code != 200:
            print(response.status_code)
            continue

        # If we already got all the businesses in a neighborhood.
        content = json.loads(response.content)
        if len(content["businesses"]) == 0:
            break

        for business in content["businesses"]:
            alias_to_content[business["alias"]] = business

In [ ]:
# Write file.
file_path = "{}/restaurants.json".format(os.getcwd())
with open(file_path, "w") as fp:
    json.dump(alias_to_content, fp)

## Create Yelp Reviews Files.
Make one file per letter so we can work in parallel.

In [ ]:
# Read file.
path = "{}/restaurants.json".format(os.getcwd())
with open(path, "r") as json_file:
    alias_to_content = json.loads(json_file.read())

In [ ]:
# Make one file per letter so we can work in parallel.
for letter in string.ascii_lowercase:
    aliases = sorted([(a, []) for a in alias_to_content.keys() if a[0] == letter])
    path = "{}/reviews/{}.json".format(os.getcwd(), letter)
    with open(path, 'w') as fp:
        json.dump(aliases, fp)
        
# And one file for all numbers and weird characters.
aliases = sorted([(a, []) for a in alias_to_content.keys() if a[0] not in string.ascii_lowercase])
path = "{}/reviews/0.json".format(os.getcwd())
with open(path, 'w') as fp:
    json.dump(aliases, fp)

## Aggregating Yelp Reviews Data
No API for this. We call it with a sketchy endpoint.

In [ ]:
LETTER = 'q'

In [ ]:
# Read file.
path = "{}/reviews/{}.json".format(os.getcwd(), LETTER)
with open(path, "r") as json_file:
    alias_to_reviews = json.loads(json_file.read())

In [ ]:
headers = {
    "Content-Type": "text/html; charset=UTF-8",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
}

In [ ]:
for alias, reviews in alias_to_reviews:
    print("Fetching for " + alias)
    
    # If we already scraped reviews, do not do it again.
    # Scrapes take a long time, so this allows us to scrape over multiple runs.
    if len(reviews) >= 50:
        continue
    
    # Gather 10 reviews per restaurant 5 times.
    for i in range(0, 50, 10):
        url = "https://www.yelp.com/biz/{}/props?start={}".format(unidecode.unidecode(alias), i)
        request = urllib.request.Request(url, headers=headers)
        
        # Sleep or else Yelp might block your IP.
        time.sleep(2)

        # Handle link errors.
        try:
            with urllib.request.urlopen(request) as response:
                if response.code != 200:
                    print("Response " + str(response.code) + " on " + alias)
                    continue
                
                try:
                    data = json.loads(response.read())["bizDetailsPageProps"]["reviewFeedQueryProps"]["reviews"]
                    for d in data:
                        reviews.append({
                            "photoCount": d["user"]["photoCount"],
                            "reviewCount": d["user"]["reviewCount"],
                            "eliteYear": d["user"]["eliteYear"],
                            "localizedDate": d["localizedDate"],
                            "comment": d["comment"],
                            "rating": d["rating"],
                        })
                        
                except:
                    continue

        except HTTPError:
            print("Link error on " + alias)
            continue

In [ ]:
# Write file.
path = "{}/reviews/{}.json".format(os.getcwd(), LETTER)
with open(path, 'w') as fp:
    json.dump(alias_to_reviews, fp)